In [80]:
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr as expr
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [81]:
rho = expr("sqrt(r**2 + a**2*cos(theta)**2)")
Delta = expr("r**2 -2*r+a**2")
Sigma = expr("sqrt((r**2+a**2)**2 -a**2*Delta*sin(theta)**2)")
alpha = expr("rho*sqrt(Delta)/Sigma")
omega = expr("2* a*r/Sigma**2")
omega_bar = expr("Sigma*sin(theta)/rho")

P = expr("r**2 + a**2 - a*b")
R = expr("P**2 - Delta*((b-a)**2+q)")
Theta = expr("q-cos(theta)**2* (b**2/sin(theta)**2-a**2)")

In [82]:
def metric_values(r, theta, phi, a):
    rho_v = rho.subs({"r": r, "a": a, "theta": theta}).evalf()
    Delta_v = Delta.subs({"r": r, "a": a}).evalf()
    Sigma_v = Sigma.subs({"r": r, "a": a, "theta": theta, "Delta": Delta_v}).evalf()
    alpha_v = alpha.subs({"rho": rho_v, "Delta": Delta_v, "Sigma": Sigma_v}).evalf()
    omega_v = omega.subs({"a": a, "r": r, "Sigma": Sigma_v}).evalf()
    omega_bar_v = omega_bar.subs({"a": a, "r": r, "Sigma": Sigma_v, "theta": theta, "rho": rho_v}).evalf()

    return {
        "r": r,
        "theta": theta,
        "phi": phi,
        "a": a,
        "rho": rho_v,
        "Delta": Delta_v,
        "Sigma": Sigma_v,
        "alpha": alpha_v,
        "omega": omega_v,
        "omega_bar": omega_bar_v,
    }

def sub_metric_values(expr, values):
    return expr.subs(values).evalf()

In [83]:
r_c = 4
theta_c = sp.pi/2
phi_c = 0
a = 0.999 


metric_values = metric_values(r_c, theta_c, phi_c, a)

In [84]:
Omega = expr("1/(a+r**1.5)")

B_r = 0 
B_theta = 0
B_phi =1
beta = omega_bar/alpha*(Omega-omega)
beta = sub_metric_values(beta, metric_values)

In [85]:
N_x = 1.0/sp.sqrt(3) 
N_y = 1.0/sp.sqrt(3) 
N_z = 1.0/sp.sqrt(3) 

_ = sub_metric_values(1-beta*N_y, metric_values)
n_Fy =  sub_metric_values( (beta-N_y)/_ , metric_values)
n_Fx =  sub_metric_values( -N_x*sp.sqrt(1-beta**2)/_ , metric_values)
n_Fz =  sub_metric_values( -N_z*sp.sqrt(1-beta**2)/_ , metric_values)

display(n_Fx, n_Fy,n_Fz)

-0.700867727986650

-0.132547560270506

-0.700867727986650

In [86]:
kappa = sp.sqrt(1-B_theta**2)
n_Fr    = B_phi/kappa*n_Fx + B_r*n_Fy+B_r*B_theta/kappa * n_Fz
n_Ftheta= B_theta*n_Fy - kappa*n_Fz
n_Fphi  = -B_r/kappa * n_Fx + B_phi*n_Fy + B_theta*B_phi/kappa*n_Fz

display(n_Fr, n_Ftheta, n_Fphi)

-0.700867727986650

0.700867727986650

-0.132547560270506

In [87]:
E_f = sub_metric_values(1/(expr("alpha")+expr("omega*omega_bar")*n_Fphi), metric_values)
p_t = -1 
p_r = sub_metric_values(E_f*rho/sp.sqrt(Delta)*n_Fr, metric_values)
p_theta = sub_metric_values(E_f*rho*n_Ftheta, metric_values)
p_phi = sub_metric_values(E_f*omega_bar*n_Fphi, metric_values)

display(p_r, p_theta, p_phi)

-1.33267663031924

3.99758586286978

-0.790597249524871

In [88]:
b = p_phi 
q = sub_metric_values(p_theta**2 +sp.cos(theta_c)**2*(b**2/sp.sin(theta_c)**2-a**2), metric_values)
display(b,q)

-0.790597249524871

15.9806927310163

In [89]:
def eval_on(expr, values):
    return expr.subs(values).evalf()

def ray_values(p_r,p_theta,b,q, metric_values: dict):
    ray_values = metric_values.copy()

    ray_values['b'] = b
    ray_values['q'] = q

    ray_values['p_r'] = p_r 
    ray_values['p_theta'] = p_theta 

    ray_values['P'] = eval_on(P, ray_values)
    ray_values['R'] = eval_on(R, ray_values)
    ray_values['Theta'] =eval_on(Theta, ray_values)
    
    return ray_values
ray = ray_values(p_r,p_theta,b,q,metric_values)

In [90]:
# Replace functions with their evaluated value 
def replace_fns(expression, values):
    for letter in [
    'rho',
    'Delta',
    'Sigma',
    'alpha',
    'P',
    'R',
    'Theta',
    ]:
        fn = sp.Function(letter)
        expression=expression.replace(fn, sp.Lambda((x), letter))

    return expression

# Get the partial derivatives with respect to b, r, theta
def partials(sym_b, sym_r=None, sym_theta=None):
    if sym_r is None and sym_theta is None: 
        sym_r = sym_b 
        sym_theta = sym_b

    return (
        sym_b.diff("b"),
        sym_r.diff("r"),
        sym_theta.diff("theta")
    )

# Dictionary of evaluated partial derivatives
partial_values = {}

# Replace partial derivatives with their values
def replace_partials(expression):
    for (old, new) in partial_values.items(): 
        expression = expression.replace(old, new)
    return expression


# Expressions for partial derivatives
partials_rho = partials(rho)
partials_Delta = partials(Delta)
partials_Sigma = partials(
    Sigma,
    Sigma.subs("Delta", "Delta(r)"),
    Sigma,
)

partials_alpha = partials(
    alpha,
    alpha.subs({"rho": "rho(r)","Delta": "Delta(r)", "Sigma": "Sigma(r)"}),
    alpha.subs({"rho": "rho(theta)", "Sigma": "Sigma(theta)"}),
)

partials_P = partials(P)
partials_R = partials(
    R.subs({"P": "P(b)"}),
    R.subs({"Delta": "Delta(r)"}),
    R,
)
partials_Theta = partials(
    Theta
)

print([
        partials_rho,
    partials_Delta,
    partials_Sigma,
    partials_alpha,
    partials_P,
    partials_R,
    partials_Theta,
])

# Evaluate partial derivatives
for (letter, exprs) in [
    ('rho', partials_rho),
    ('Delta', partials_Delta),
    ('Sigma', partials_Sigma),
    ('alpha', partials_alpha),
    ('P', partials_P),
    ('R', partials_R),
    ('Theta', partials_Theta)
]:
    for (x, expression) in zip(['b', 'r', 'theta'], exprs):
        fn = sp.Function(letter)(x)
        dfn = fn.diff(x)

        expression= replace_partials(expression)
        expression = replace_fns(expression, ray)

        partial_values[dfn] = eval_on(expression, ray)



[(0, r/sqrt(a**2*cos(theta) + r**2), -a**2*sin(theta)/(2*sqrt(a**2*cos(theta) + r**2))), (0, 2*r - 2, 0), (0, (-a**2*sin(theta)**2*Derivative(Delta(r), r)/2 + 2*r*(a**2 + r**2))/sqrt(-a**2*Delta(r)*sin(theta)**2 + (a**2 + r**2)**2), -Delta*a**2*sin(theta)*cos(theta)/sqrt(-Delta*a**2*sin(theta)**2 + (a**2 + r**2)**2)), (0, sqrt(Delta(r))*Derivative(rho(r), r)/Sigma(r) - sqrt(Delta(r))*rho(r)*Derivative(Sigma(r), r)/Sigma(r)**2 + rho(r)*Derivative(Delta(r), r)/(2*sqrt(Delta(r))*Sigma(r)), sqrt(Delta)*Derivative(rho(theta), theta)/Sigma(theta) - sqrt(Delta)*rho(theta)*Derivative(Sigma(theta), theta)/Sigma(theta)**2), (-a, 2*r, 0), (-Delta*(-2*a + 2*b) + 2*P(b)*Derivative(P(b), b), -(q + (-a + b)**2)*Derivative(Delta(r), r), 0), (-2*b*cos(theta)**2/sin(theta)**2, 0, 2*b**2*cos(theta)**3/sin(theta)**3 + 2*(-a**2 + b**2/sin(theta)**2)*sin(theta)*cos(theta))]


In [91]:
def subs_partial(expression):
    expression = replace_partials(expression)
    expression = replace_fns(expression, ray)
    expression = eval_on(expression, ray)
    return expression

d_r = expr("Delta/rho**2*p_r")
d_theta = expr("1/rho**2*p_theta")

d_phi = expr("(R(b)+Delta*Theta(b))/(2*Delta*rho**2)").diff("b")
d_p_r     = sp.diff(expr("-Delta(r)/(2*rho(r)**2)*p_r**2-1/(2*rho(r)**2)*p_theta**2+((R+Delta(r)*Theta)/(2*Delta(r)*rho(r)**2))"),"r")
d_p_theta = sp.diff(expr("-Delta/(2*rho(theta)**2)*p_r**2-1/(2*rho(theta)**2)*p_theta**2+((R(theta)+Delta(theta)*Theta(theta))/(2*Delta*rho(theta)**2))"),"theta")

subs_partial(d_r)
subs_partial(d_theta)
subs_partial(d_phi)
subs_partial(d_p_r)
subs_partial(d_p_theta)

-0.749464103268070

0.249849116429361

-0.0115805027402825

-0.666010125374639

5.20417042793042e-17

In [92]:
data = (
    [],[],[]
)

for i in range(100):
    print(i)
    h = 0.01
    new_state = (
        (state[0] + h*sub_state(d_r, state)).evalf(),
        (state[1] + h*sub_state(d_theta, state)).evalf(),
        (state[2] + h*sub_state(d_phi, state)).evalf(),
        (state[3] + h*sub_state(d_p_r, state)).evalf(),
        (state[4] + h*sub_state(d_p_theta, state)).evalf(),
    )
    state = new_state
    _r=state[0]
    _theta=state[1]
    _phi=state[2]
    data[0].append((sqrt(_r**2+a**2)*sin(_theta)*cos(_phi)).evalf())
    data[1].append((sqrt(_r**2+a**2)*sin(_theta)*sin(_phi)).evalf())
    data[2].append((_r*cos(_theta)).evalf())


0


NameError: name 'state' is not defined

In [ ]:
print(state[0])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


ax = plt.figure().add_subplot(projection='3d')

x = data[0]
y = data[1]
z = data[2]

ax.plot(x, y, z, label='parametric curve')
ax.legend()

plt.show()

In [ ]:
data